# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=AccountLiquidator.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 AccountLiquidator

An `AccountLiquidator` takes a `MarginAccount` and liquidates it (if possible).

In [ ]:
import logging
import typing

from decimal import Decimal
from solana.transaction import Transaction

from BaseModel import Group, MarginAccount
from Context import Context
from Instructions import ForceCancelOrdersInstructionBuilder, InstructionBuilder, LiquidateInstructionBuilder
from Wallet import Wallet


# 💧 AccountLiquidator class

This base class takes a `MarginAccount` and liquidates it.

Derived classes may override `prepare_instructions()` to extend the liquidation process (for example to cancel outstanding orders before liquidating).

In [ ]:
class AccountLiquidator:
    def __init__(self, context: Context, wallet: Wallet, group: Group):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)
        self.context = context
        self.wallet = wallet
        self.group = group

    def prepare_instructions(self, margin_account: MarginAccount, prices: typing.List[Decimal]) -> typing.List[InstructionBuilder]:
        liquidate_instructions: typing.List[InstructionBuilder] = []
        liquidate_instruction = LiquidateInstructionBuilder.from_margin_account_and_market(self.context, self.group, self.wallet, margin_account, prices)
        if liquidate_instruction is not None:
            liquidate_instructions += [liquidate_instruction]

        return liquidate_instructions

    def liquidate(self, margin_account: MarginAccount, prices: typing.List[Decimal]) -> typing.Optional[str]:
        instructions = self.prepare_instructions(margin_account, prices)

        if len(instructions) == 0:
            return None

        transaction = Transaction()
        for instruction in instructions:
            transaction.add(instruction.build())

        for instruction in transaction.instructions:
            self.logger.debug("INSTRUCTION")
            self.logger.debug("    Keys:")
            for key in instruction.keys:
                self.logger.debug("        ", f"{key.pubkey}".ljust(45), f"{key.is_signer}".ljust(6), f"{key.is_writable}".ljust(6))
            self.logger.debug("    Data:", " ".join(f"{x:02x}" for x in instruction.data))
            self.logger.debug("    Program ID:", instruction.program_id)

        transaction_response = self.context.client.send_transaction(transaction, self.wallet.account)
        return transaction_response["result"]



# 🌪️ ForceCancelOrdersAccountLiquidator class

When liquidating an account, it's a good idea to ensure it has no open orders that could lock funds. This is why Mango allows a liquidator to force-close orders on a liquidatable account.

`ForceCancelOrdersAccountLiquidator` overrides `prepare_instructions()` to inject any necessary force-cancel instructions before the `PartialLiquidate` instruction.

This is not always necessary. For example, if the liquidator is partially-liquidating a large account, then perhaps only the first partial-liquidate needs to check and force-close orders, and subsequent partial liquidations can skip this step as an optimisation.

The separation of the regular `AccountLiquidator` and the `ForceCancelOrdersAccountLiquidator` classes allows the caller to determine which process is used.

In [ ]:
class ForceCancelOrdersAccountLiquidator(AccountLiquidator):
    def __init__(self, context: Context, wallet: Wallet, group: Group):
        super().__init__(context, wallet, group)

    def prepare_instructions(self, margin_account: MarginAccount, prices: typing.List[Decimal]) -> typing.List[InstructionBuilder]:
        force_cancel_orders_instructions: typing.List[InstructionBuilder] = []
        for index, market_metadata in enumerate(self.group.markets):
            open_orders = margin_account.open_orders_accounts[index]
            if open_orders is not None:
                market = market_metadata.fetch_market(self.context)
                orders = market.load_orders_for_owner(margin_account.owner)
                order_count = len(orders)
                if order_count > 0:
                    force_cancel_orders_instructions += ForceCancelOrdersInstructionBuilder.multiple_instructions_from_margin_account_and_market(self.context, self.group, self.wallet, margin_account, market_metadata, order_count)

        all_instructions = force_cancel_orders_instructions + super().prepare_instructions(margin_account, prices)

        return all_instructions


# 🏃 Running

In [ ]:
if __name__ == "__main__":
    logging.getLogger().setLevel(logging.INFO)

    from Context import default_context
    from Wallet import default_wallet

    if default_wallet is None:
        print("No default wallet file available.")
    else:
        group = Group.load(default_context)
        prices = group.get_prices()
        margin_accounts = MarginAccount.load_all_for_owner(default_context, default_wallet.address, group)
        margin_account = margin_accounts[0]

        account_liquidator = AccountLiquidator(default_context, default_wallet, group)
        print(account_liquidator.prepare_instructions(margin_account, prices))

        force_cancel_orders_account_liquidator = ForceCancelOrdersAccountLiquidator(default_context, default_wallet, group)
        print(force_cancel_orders_account_liquidator.prepare_instructions(margin_account, prices))
